In [1]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
from bson.son import SON
import folium 
import math
import numpy as np
from IPython.display import display
import bus 

In [80]:
#Estableciendo la conexion con la base de datos morte_motoquiro en MongoDB
db = MongoClient("10.0.0.107", 27017).feria_livro_event

In [3]:
#Important: In geoWithin function, if you use longitude and latitude, specify longitude first.
#funcion que recibe un poligono y una coleccion de Mongo y te devuelve todos los puntos de la linea que aquel dia pasaron por aquel poligono
def points_inside_polygon(mongo_collection, inverted_polygon):
    points_inside_polygon = []
    for row in mongo_collection.find ({"latlon": {"$geoWithin": { "$polygon": inverted_polygon }}}).sort("timestamp",pymongo.ASCENDING):
        points_inside_polygon.insert(len(points_inside_polygon), [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'line'], row[u'bus_id']]) 
    df_points_inside_polygon = pd.DataFrame(points_inside_polygon, columns=['timestamp', 'latitude', 'longitude', 'line', 'bus_id'])
    return df_points_inside_polygon

In [4]:
#Poligono tramo 2-5
Punto2 = (-22.960298,-43.204346)
Punto5 = (-22.974325,-43.22625)
#inverted_polygon_2_5
T = (-22.974577,-43.226631)
inverted_T = (-43.226631,-22.974577)
V = (-22.974654,-43.225901)
inverted_V = (-43.225901,-22.974654)
W = (-22.961393,-43.215076)
inverted_W = (-43.215076,-22.961393)
X = (-22.961897,-43.214658)
inverted_X = (-43.214658,-22.961897)
Y = (-22.960128,-43.2041)
inverted_Y = (-43.2041,-22.960128)
Z = (-22.960484,-43.204015)
inverted_Z = (-43.204015,-22.960484)
inverted_polygon_2_5 = [inverted_V,inverted_T,inverted_W,inverted_Y,inverted_Z,inverted_X]

In [56]:
inverted_polygon_2_5

[(-43.225901, -22.974654),
 (-43.226631, -22.974577),
 (-43.215076, -22.961393),
 (-43.2041, -22.960128),
 (-43.204015, -22.960484),
 (-43.214658, -22.961897)]

In [5]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [83]:
Polygon1 = [
    [-22.967817543529037, -43.39148998260498],
    [-22.970089533152084, -43.390910625457764],
    [-22.973210987056486, -43.38676929473877],
    [-22.973131963796394, -43.38468790054321],
    [-22.969180741885026, -43.38968753814697],
    [-22.967284114343567, -43.390052318573],
    [-22.967817543529037, -43.39148998260498]
    ]


Polygon2 = [(-22.972144169146848, -43.37882995605469),
            (-22.974356819135682, -43.37818622589111),
            (-22.97423828523316, -43.38724136352539),
            (-22.97419877390921, -43.398399353027344),
            (-22.973645614161107, -43.4020471572876),
            (-22.9730529404901, -43.403892517089844),
            (-22.970800756838422, -43.40178966522217),
            (-22.972104657210703, -43.39792728424072),
            (-22.972381240521123, -43.38719844818115),
           (-22.972144169146848, -43.37882995605469)]

Polygon3=[
            (-22.972381240521123, -43.38050365447998),
            (-22.97419877390921, -43.3800745010376),
            (-22.97423828523316, -43.36329460144043),
            (-22.97234172865428, -43.36355209350586),
            (-22.972381240521123, -43.38050365447998)]

Polygon4=[(-22.9772806224873, -43.418312072753906),
          (-22.966612385708647, -43.413848876953125),
          (-22.962740002623352, -43.405609130859375),
          (-22.959499760132136, -43.392391204833984),
          (-22.961870676892556, -43.39067459106445),
          (-22.964952806537738, -43.4044075012207),
          (-22.96842999666493, -43.41118812561035),
          (-22.97894005030228, -43.415822982788086),
         (-22.9772806224873, -43.418312072753906)]

Polygon5=[(-22.978189359291218, -43.41747522354126),
          (-22.977497929670303, -43.41678857803345),
          (-22.982377371658988, -43.4105658531189),
          (-22.98314779375871, -43.411102294921875),
         (-22.978189359291218, -43.41747522354126)]

Polygon6=[(-23.00651517560899, -43.43925476074219),
          (-23.00817424418687, -43.43625068664551),
          (-22.999246634752332, -43.42629432678223),
          (-22.998140518132615, -43.423805236816406),
          (-22.98984435460933, -43.41470718383789),
          (-22.983839194466338, -43.4099006652832),
          (-22.981784736284872, -43.411617279052734),
          (-22.986446731008904, -43.41547966003418),
          (-22.994664092609916, -43.42526435852051),
          (-22.996086277507416, -43.4274959564209),
         (-23.00651517560899, -43.43925476074219)]


# Create the map and add the line
m=folium.Map(location=[-22.972144169146848, -43.37882995605469], zoom_start=14)
m.line(Polygon1, line_color='#FF0000', line_weight=5)
m.line(Polygon2, line_color='blue', line_weight=5)
m.line(Polygon3, line_color='green', line_weight=5)
m.line(Polygon4, line_color='brown', line_weight=5)
m.line(Polygon5, line_color='magenta', line_weight=5)
m.line(Polygon6, line_color='black', line_weight=5)

embed_map(m)


In [69]:
def invert_polygon(polygon):
    polygon_inverted = []
    for points in polygon:
        polygon_inverted.insert(1, (points[1],points[0]))
    return polygon_inverted

In [84]:
Polygon1_inverted = invert_polygon(Polygon1)
Polygon2_inverted = invert_polygon(Polygon2)
Polygon3_inverted = invert_polygon(Polygon3)
Polygon4_inverted = invert_polygon(Polygon4)
Polygon5_inverted = invert_polygon(Polygon5)
Polygon6_inverted = invert_polygon(Polygon6)

In [85]:
points_inside_polygon1 = points_inside_polygon(db.dados20150905, Polygon1_inverted)
points_inside_polygon2 = points_inside_polygon(db.dados20150905, Polygon2_inverted)
points_inside_polygon3 = points_inside_polygon(db.dados20150905, Polygon3_inverted)
points_inside_polygon4 = points_inside_polygon(db.dados20150905, Polygon4_inverted)
points_inside_polygon5 = points_inside_polygon(db.dados20150905, Polygon5_inverted)
points_inside_polygon6 = points_inside_polygon(db.dados20150905, Polygon6_inverted)

In [129]:
for doc in points_inside_polygon6.iterrows():
    m.polygon_marker( [ doc[1]['latitude'], doc[1]['longitude'] ], fill_color='black', num_sides=4, radius=6)
embed_map(m)

In [128]:
points_inside_polygon6['line'].unique()

array([u'315', u'352', u'360', u'382', u'613', u'2335', u'525', u'2329',
       u'LECD10', u'368', u'348', u'361', u'880', u'2333', u'2803',
       u'2338', u'332', u'2804', u'306', u'2334', u'614', u'887', u'2330',
       u'2115', u'863', u'612', u'692'], dtype=object)